## End to end Deep Learning Project Using GRU RNN

In [1]:
import pandas as pd
import numpy as np
import re
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, Embedding, Input
from tensorflow.keras.utils import to_categorical
import nltk

In [2]:
def load_stopwords(file_path):
    with open(file_path, 'r') as file:
        stopwords = file.read().splitlines()
    return set(stopwords)

stopwords = load_stopwords('english_stopwords.txt')


In [3]:
df = pd.read_csv('data.csv', encoding='latin-1', header=None)
df.columns = ['polarity', 'id', 'date', 'query', 'user', 'tweet']

In [4]:
df = df[['polarity', 'tweet']]
df['polarity'] = df['polarity'].map({0: 0, 4: 1}) 

In [5]:
df.shape

(1600000, 2)

In [6]:
total_samples = 20000

class_counts = df['polarity'].value_counts()
min_class_count = class_counts.min() 
samples_per_class = total_samples // 2 
samples_per_class = min(samples_per_class, min_class_count)

In [7]:
balanced_df = pd.concat([
    df[df['polarity'] == 0].sample(samples_per_class, random_state=42),
    df[df['polarity'] == 1].sample(samples_per_class, random_state=42)
])

balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)
print(balanced_df['polarity'].value_counts())
print(f"Total samples in balanced subset: {len(balanced_df)}")

polarity
1    10000
0    10000
Name: count, dtype: int64
Total samples in balanced subset: 20000


In [8]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Handle negations more effectively
    negations = ["not", "no", "never", "none", "n't"]
    words = text.split()
    
    # Create a flag to indicate if the current word is negated
    negation_flag = False
    processed_words = []

    for word in words:
        if word in negations:
            negation_flag = True
        else:
            if negation_flag:
                processed_words.append("not_" + word)  # Prefix 'not_' to the word
                negation_flag = False  # Reset the flag after the first word
            else:
                processed_words.append(word)

    text = ' '.join(processed_words)

    # Remove URLs, mentions, hashtags, punctuation, and numbers
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'[^a-z\s]', '', text)

    # Remove stopwords
    tokens = text.split()
    tokens = [word for word in tokens if word not in stopwords]
    return ' '.join(tokens)

In [9]:
balanced_df['tweet'] = balanced_df['tweet'].apply(preprocess_text)

In [10]:
max_length = balanced_df['tweet'].apply(lambda x: len(x.split())).max()
print("Maximum tweet length:", max_length)

Maximum tweet length: 25


In [11]:
vocab_size = 10000
dimensions = 50

In [12]:
X_train, X_test, y_train, y_test = train_test_split(balanced_df['tweet'], balanced_df['polarity'], test_size=0.2, random_state=42)

In [13]:
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

In [14]:
with open('tokenizer.pkl', 'wb') as file:
    pickle.dump(tokenizer, file)

In [15]:
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='pre')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding='pre')

In [16]:
num_classes = 2
y_train_encoded = to_categorical(y_train, num_classes)
y_test_encoded = to_categorical(y_test, num_classes)

In [17]:
def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

In [18]:
def create_embedding_matrix(embeddings_index, tokenizer):
    embedding_matrix = np.zeros((vocab_size, dimensions))
    for word, i in tokenizer.word_index.items():
        if i < vocab_size:
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [19]:
glove_file_path = 'glove.6B.50d.txt'
embeddings_index = load_glove_embeddings(glove_file_path)
embedding_matrix = create_embedding_matrix(embeddings_index, tokenizer)

In [20]:
model = Sequential()
model.add(Input(shape=(max_length,)))
model.add(Embedding(vocab_size, dimensions, weights=[embedding_matrix], trainable=False, input_length=max_length))
model.add(GRU(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(100))
model.add(Dense(num_classes, activation="softmax")) 
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
model.summary()

c:\ProgramData\Anaconda11\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 25, 50)         │       500,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 25, 150)        │        90,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 25, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 100)            │        75,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │           202 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 666,702 (2.54 MB)

 Trainable params: 166,702 (651.18 KB)

 Non-trainable params: 500,000 (1.91 MB)

In [21]:
## Create an instance of EarlyStoppping Callback
from tensorflow.keras.callbacks import EarlyStopping
earlystopping=EarlyStopping(monitor='val_loss',patience=8,restore_best_weights=True)
earlystopping

In [22]:
model.fit(X_train_padded, y_train_encoded, validation_split=0.2, epochs=20, callbacks=[earlystopping])

Epoch 1/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 58s 129ms/step - accuracy: 0.6329 - loss: 0.6446 - val_accuracy: 0.6794 - val_loss: 0.5976
Epoch 2/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 39s 97ms/step - accuracy: 0.6791 - loss: 0.5910 - val_accuracy: 0.6894 - val_loss: 0.5752
Epoch 3/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 31s 78ms/step - accuracy: 0.6928 - loss: 0.5714 - val_accuracy: 0.7056 - val_loss: 0.5619
Epoch 4/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 30s 75ms/step - accuracy: 0.7117 - loss: 0.5486 - val_accuracy: 0.7109 - val_loss: 0.5592
Epoch 5/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 30s 74ms/step - accuracy: 0.7363 - loss: 0.5162 - val_accuracy: 0.7081 - val_loss: 0.5600
Epoch 6/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 30s 75ms/step - accuracy: 0.7681 - loss: 0.4732 - val_accuracy: 0.7097 - val_loss: 0.5642
Epoch 7/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 32s 80ms/step - accuracy: 0.7872 - loss: 0.4434 - val_accuracy: 0.7069 - val_loss: 0.6024
Epoch 8/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 36s 90ms/step - accuracy: 0.8178 - loss: 0.3905 -

In [23]:
## Save model file
model.save('gru_rnn.keras')